In [1]:
import sys
import time
from charm.toolbox.eccurve import secp256k1, secp192k1, secp160k1
from blindIssuing_ecc import initialization, tracer_choose_keypair, Issuer, User, verify
import pandas as pd

In [2]:
L = secp256k1
round = 10
m = b'my msg'

### Issuing Test

In [3]:
def issue_test(L, round, time_recoder=[], flag=1):
    params = initialization(L)
    traceKeypair = tracer_choose_keypair(params)
    tkey = traceKeypair.yt
    xt = traceKeypair.xt
    total = 0
    for x in range(round):
        start = time.time()
        issuer = Issuer(params,tkey)
        # get the hash 
        issuer.start()

        user = User(params, issuer.IssuerKeypair.y,tkey)    
        user.start()

        zu, xi = user.protocol_one()
        z1, a, b1, b2 = issuer.protocol_two(zu,xi)    
        e = user.protocol_three(z1, a, b1, b2, m)    
        r, c, s1, s2, d = issuer.protocol_four(e)    
        roi, pi, sigma1, sigma2, delta = user.protocol_five(r, c, s1, s2, d)

        end = time.time()
        runtime = end - start
        if not flag: time_recoder.append(runtime)
        msg = '{func} takes {time} second to complete'
        total = total + runtime
        if flag: print(msg.format(func = '[Credential Issuing]', time = runtime))
    if flag: print("The average time is {time} second for {rounds} tests".format(time=total/round, rounds=round))

In [4]:
issue_test(L, round)

[Credential Issuing] takes 0.007844686508178711 second to complete
[Credential Issuing] takes 0.007528066635131836 second to complete
[Credential Issuing] takes 0.007661581039428711 second to complete
[Credential Issuing] takes 0.007700920104980469 second to complete
[Credential Issuing] takes 0.007668972015380859 second to complete
[Credential Issuing] takes 0.007695198059082031 second to complete
[Credential Issuing] takes 0.007824897766113281 second to complete
[Credential Issuing] takes 0.007707834243774414 second to complete
[Credential Issuing] takes 0.007971763610839844 second to complete
[Credential Issuing] takes 0.007707834243774414 second to complete
The average time is 0.007731175422668457 second for 10 tests


---

### Parameter Test

In [5]:
def parameter_test(L, round, time_recoder=[], flag=1):
    total = 0
    for x in range(round):
        start = time.time()
        params = initialization(L)
        traceKeypair = tracer_choose_keypair(params)
        tkey = traceKeypair.yt
        xt = traceKeypair.xt
        end = time.time()
        runtime = end - start
        if not flag:time_recoder.append(runtime)
        msg = '{func} takes {time} second to complete'
        total = total + runtime
        if flag: print(msg.format(func = '[Parameter Generation]', time = runtime))
    if flag: print("The average time is {time} second for {rounds} tests".format(time=total/round, rounds=round))

In [6]:
parameter_test(L, round)

[Parameter Generation] takes 0.0008521080017089844 second to complete
[Parameter Generation] takes 0.0008642673492431641 second to complete
[Parameter Generation] takes 0.0008549690246582031 second to complete
[Parameter Generation] takes 0.0008461475372314453 second to complete
[Parameter Generation] takes 0.0008308887481689453 second to complete
[Parameter Generation] takes 0.0009164810180664062 second to complete
[Parameter Generation] takes 0.0008661746978759766 second to complete
[Parameter Generation] takes 0.0008342266082763672 second to complete
[Parameter Generation] takes 0.0008628368377685547 second to complete
[Parameter Generation] takes 0.0008618831634521484 second to complete
The average time is 0.0008589982986450196 second for 10 tests


---

### Verifying Test

In [7]:
def verifying_test(L, round, time_recoder=[], flag=1):
    total = 0
    params = initialization(L)
    traceKeypair = tracer_choose_keypair(params)
    tkey = traceKeypair.yt
    xt = traceKeypair.xt
    
    for x in range(round):
        issuer = Issuer(params, tkey)
        issuer.start()

        user = User(params, issuer.IssuerKeypair.y,tkey)    
        user.start()

        zu, xi = user.protocol_one()    
        z1, a, b1, b2 = issuer.protocol_two(zu,xi)    
        e = user.protocol_three(z1, a, b1, b2, m)    
        r, c, s1, s2, d = issuer.protocol_four(e)    
        roi, pi, sigma1, sigma2, delta = user.protocol_five(r, c, s1, s2, d)
        
        start = time.time()
        verify(roi, pi, sigma1, sigma2, delta, params, m, user.y, user.zeta1,user.zeta2, user.z)
        end = time.time()
        runtime = end - start
        if not flag: time_recoder.append(runtime)
        msg = '{func} takes {time} second to complete'
        total = total + runtime
        if flag: print(msg.format(func = '[Credential Verifying]', time = runtime))
    if flag: print("The average time is {time} second for {rounds} tests".format(time=total/round, rounds=round))

In [8]:
verifying_test(L, round)

[Credential Verifying] takes 0.0024726390838623047 second to complete
[Credential Verifying] takes 0.0024449825286865234 second to complete
[Credential Verifying] takes 0.002430438995361328 second to complete
[Credential Verifying] takes 0.0024261474609375 second to complete
[Credential Verifying] takes 0.002432107925415039 second to complete
[Credential Verifying] takes 0.002431631088256836 second to complete
[Credential Verifying] takes 0.0024340152740478516 second to complete
[Credential Verifying] takes 0.002429485321044922 second to complete
[Credential Verifying] takes 0.002428293228149414 second to complete
[Credential Verifying] takes 0.0024344921112060547 second to complete
The average time is 0.0024364233016967774 second for 10 tests


---

In [9]:
times = 1000
issue_time = []
parameter_time = []
verify_time = []
identify = ['Average Time']
identify.extend(['#'+str(x) for x in range(1,times + 1)])
issue_test(L, times, issue_time, 0)
parameter_test(L, times, parameter_time, 0)
verifying_test(L, times, verify_time, 0)
avg1 = sum(issue_time)/times
avg2 = sum(parameter_time)/times
avg3 = sum(verify_time)/times
issue_time.insert(0, avg1)
parameter_time.insert(0, avg2)
verify_time.insert(0, avg3)
dataframe = pd.DataFrame({'Rounds [TEST DATE: ' + time.strftime('%Y-%m-%d %H:%M',time.localtime(time.time())) + ']':identify, 'Issuing Time':issue_time, 'Parameter Time': parameter_time, 'Verifying Time': verify_time})
dataframe.to_csv('./ECC_256_result.csv', index=False)